# Encodeur Transformer

![Transformers](./asset/trasnformers_encoder_decoder.png)

Tout d'abord, nous allons examiner l'architecture du Transformer, mais avec uniquement un encodeur.

Voici un exemple **simplifié** d’un *Transformer Decoder-Only* (type GPT). L’objectif est de montrer la différence principale avec un *Encoder-Only* : ici, chaque couche applique une *causal self-attention* (unidirectionnelle) qui empêche un token de regarder les tokens futurs. Il n’y a ni encodeur séparé ni cross-attention.

Nous allons reprendre l’ossature générale (embeddings, attention multi-tête, feed-forward, etc.) en adaptant :
- **Le masque** : on utilise un *causal mask* (triangulaire) plutôt qu’un masque bidirectionnel ou de padding.
- **Les couches** : on n’a pas de cross-attention, seulement une couche de *masked (causal) self-attention* suivie d’un feed-forward.

![decoder-only](./asset/decoder_only_mask.png)

---

# 2. Différences clés entre *Encoder-Only* et *Decoder-Only*

1. **Masque d’attention** :
   - **Encoder-Only (BERT)** : il peut s’attendre sur tout le contexte (bidirectionnel), mais masque souvent des tokens de padding ou masqués.
   - **Decoder-Only (GPT)** : causal (unidirectionnel). Chaque token ne peut voir que les tokens précédents (positions antérieures).

2. **Architecture** :
   - **Encoder-Only (BERT)** : on empile des blocs d’encodeur.
   - **Decoder-Only (GPT)** : on empile des blocs de *decoder* qui, en version la plus simple, n’utilise pas de cross-attention (puisqu’il n’y a pas d’encodeur). On y retrouve tout de même la couche de *self-attention* et le feed-forward.

3. **Utilisation en génération** :
   - **Encoder-Only** : pour la classification, l’extraction de features ou le *masked language modeling*.
   - **Decoder-Only** : pour générer du texte (autocomplétion), car il se base sur l’historique (les tokens déjà générés).

![summary](./asset/summary.png)



# 1. Imports

In [ ]:
import torch
import torch.nn as nn
import math

---

# 3. Embeddings

Comme pour BERT, on garde un embedding de tokens et un embedding de positions.
Dans GPT, on ne gère généralement pas de *token_type_embeddings*.

In [ ]:
class GPTEmbeddings(nn.Module):
    def __init__(self, vocab_size, hidden_size, max_position_embeddings=512):
        super().__init__()
        self.word_embeddings = nn.Embedding(vocab_size, hidden_size)
        self.position_embeddings = nn.Embedding(max_position_embeddings, hidden_size)

        self.layer_norm = nn.LayerNorm(hidden_size, eps=1e-5)
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, input_ids):
        """
        input_ids: [batch_size, seq_length]
        """
        batch_size, seq_length = input_ids.size()

        # Positions: [0..seq_length-1]
        position_ids = torch.arange(seq_length, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand(batch_size, seq_length)

        # Embeddings
        token_embeddings = self.word_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)

        hidden_states = token_embeddings + position_embeddings
        hidden_states = self.layer_norm(hidden_states)
        hidden_states = self.dropout(hidden_states)

        return hidden_states

# 4. Causal Multi-Head Self-Attention

La seule différence majeure avec la *self-attention* classique se situe au niveau du *masque* : on applique un masque triangulaire pour interdire l’attention sur les tokens futurs.

In [ ]:
class CausalMultiHeadSelfAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super().__init__()
        assert hidden_size % num_heads == 0, "hidden_size doit être divisible par num_heads."

        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads

        self.query = nn.Linear(hidden_size, hidden_size)
        self.key   = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)

        self.out = nn.Linear(hidden_size, hidden_size)

    def forward(self, hidden_states, attention_mask=None):
        """
        hidden_states: [batch_size, seq_length, hidden_size]
        attention_mask: [batch_size, 1, seq_length, seq_length] ou None
                        Ici, on va principalement se concentrer sur le "causal mask"
        """
        batch_size, seq_length, _ = hidden_states.size()

        # Projections linéaires
        Q = self.query(hidden_states)
        K = self.key(hidden_states)
        V = self.value(hidden_states)

        # On reshape pour isoler les têtes
        Q = Q.view(batch_size, seq_length, self.num_heads, self.head_dim)
        K = K.view(batch_size, seq_length, self.num_heads, self.head_dim)
        V = V.view(batch_size, seq_length, self.num_heads, self.head_dim)

        # On transpose pour avoir [batch_size, num_heads, seq_length, head_dim]
        Q = Q.transpose(1, 2)
        K = K.transpose(1, 2)
        V = V.transpose(1, 2)

        # Calcul des scores d'attention : Q * K^T / sqrt(dim)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        # => [batch_size, num_heads, seq_length, seq_length]

        # Masque causal : on veut annuler (ou fortement pénaliser) l'attention sur les positions futures.
        # On peut créer un masque triangulaire inférieur : shape (seq_length, seq_length)
        # True ou 1.0 => autorisé, False ou 0.0 => bloqué
        # On suppose un masque [seq_length, seq_length] = 1 pour i >= j et 0 sinon
        # On peut le construire et l'ajouter comme un grand score négatif.
        causal_mask = torch.tril(torch.ones((seq_length, seq_length), device=hidden_states.device)).view(1, 1, seq_length, seq_length)

        # Si on a un attention_mask externe (pour du padding par exemple), on le combine
        # L'idée: on veut que la position i ne voie pas la position j > i, et en plus
        # on peut masquer certains tokens.
        # On fusionne : causal_mask ET attention_mask s'ils sont donnés
        # Généralement, on multiplie ou on additionne avec un log prob négatif.

        if attention_mask is not None:
            # attention_mask est typiquement [batch_size, 1, seq_length, seq_length]
            mask = causal_mask * attention_mask
        else:
            mask = causal_mask

        # mask: 1 => autorisé, 0 => bloqué
        # On convertit en "score" (ici on utilise -1e9 pour bloquer)
        mask_value = -1e9
        scores = scores.masked_fill(mask == 0, mask_value)

        # Softmax
        attn_weights = torch.softmax(scores, dim=-1)  # [batch_size, num_heads, seq_length, seq_length]

        # On pèse les V
        context = torch.matmul(attn_weights, V)  # [batch_size, num_heads, seq_length, head_dim]

        # On remet la forme [batch_size, seq_length, hidden_size]
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_length, self.num_heads * self.head_dim)

        # Projection finale
        output = self.out(context)
        return output

> **Remarque** : Dans un *Decoder-Only Transformer*, on applique ce masque causal à chaque couche. Ceci assure qu’un token ne peut pas dépendre d’un token futur.

---

# 5. Feed-Forward

Inchangé par rapport à BERT (on utilise souvent GELU ou ReLU).

In [ ]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, hidden_size, intermediate_size):
        super().__init__()
        self.fc1 = nn.Linear(hidden_size, intermediate_size)
        self.fc2 = nn.Linear(intermediate_size, hidden_size)
        self.activation = nn.GELU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

# 6. Un bloc de Decoder-Only

Un bloc (couche) *Decoder-Only* simple comprend :
1. Causal *Self-Attention* + Add & LayerNorm
2. Feed Forward + Add & LayerNorm

(Comparé au *Decoder Transformer* standard dans le papier original, il peut y avoir une partie cross-attention si on décode depuis un encodeur, mais dans GPT-like, on n’en a pas).

In [ ]:
class GPTBlock(nn.Module):
    def __init__(self, hidden_size, num_heads, intermediate_size):
        super().__init__()
        self.attn = CausalMultiHeadSelfAttention(hidden_size, num_heads)
        self.attn_layer_norm = nn.LayerNorm(hidden_size, eps=1e-5)

        self.ff = PositionwiseFeedForward(hidden_size, intermediate_size)
        self.ff_layer_norm = nn.LayerNorm(hidden_size, eps=1e-5)

        self.dropout = nn.Dropout(p=0.1)

    def forward(self, hidden_states, attention_mask=None):
        # Causal Self-Attention
        attn_output = self.attn(hidden_states, attention_mask=attention_mask)
        hidden_states = self.attn_layer_norm(hidden_states + self.dropout(attn_output))

        # Feed-Forward
        ff_output = self.ff(hidden_states)
        hidden_states = self.ff_layer_norm(hidden_states + self.dropout(ff_output))

        return hidden_states

# 7. Le Decoder-Only complet

On empile plusieurs `GPTBlock`.

In [ ]:
class SimpleGPTModel(nn.Module):
    def __init__(self,
                 vocab_size=30522,
                 hidden_size=128,
                 num_heads=4,
                 num_layers=4,
                 intermediate_size=256,
                 max_position_embeddings=512):
        super().__init__()

        # Embeddings
        self.embeddings = GPTEmbeddings(vocab_size, hidden_size, max_position_embeddings)

        # Bloc(s) GPT
        self.blocks = nn.ModuleList([
            GPTBlock(hidden_size, num_heads, intermediate_size)
            for _ in range(num_layers)
        ])

        # Tête de sortie (ex: LM Head)
        # Le plus souvent, GPT partage le poids de self.word_embeddings
        # avec cette couche finale. Pour la démo, on fait juste un linear.
        self.lm_head = nn.Linear(hidden_size, vocab_size, bias=False)

    def forward(self, input_ids, attention_mask=None):
        """
        input_ids: [batch_size, seq_length]
        attention_mask: [batch_size, 1, seq_length, seq_length] (optionnel, ex. pour padding)
        """

        # Embeddings
        hidden_states = self.embeddings(input_ids)

        # On envoie dans chaque bloc
        for block in self.blocks:
            hidden_states = block(hidden_states, attention_mask=attention_mask)

        # Projection finale vers les logits
        logits = self.lm_head(hidden_states)

        # logits: [batch_size, seq_length, vocab_size]
        return logits

---

# 8. Exemple d’utilisation

On crée un *batch* fictif.


In [ ]:
batch_size = 2
seq_length = 6

model = SimpleGPTModel(
    vocab_size=1000,
    hidden_size=128,
    num_heads=4,
    num_layers=2,
    intermediate_size=256,
    max_position_embeddings=512
)

# Données factices
input_ids = torch.randint(0, 1000, (batch_size, seq_length))

# Exemple de masque d’attention (1=token valide, 0=padding)
# Pour la causalité, on va en plus générer le masque triangulaire (à l’intérieur du module).
attention_mask = torch.ones(batch_size, seq_length, dtype=torch.long)  # pas de padding ici
# On convertit en [batch_size, 1, seq_length, seq_length] pour rester cohérent
attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)  # => [2, 1, 1, 6]
attention_mask = attention_mask.expand(-1, -1, seq_length, -1)  # => [2, 1, 6, 6]
# => 1 partout => pas de blocage (sauf causal). On aurait pu laisser None.

with torch.no_grad():
    logits = model(input_ids, attention_mask=attention_mask)

print("Shape des logits :", logits.shape)
# => [2, 6, 1000]

---

## Résumé des différences majeures

1. **Masque causal (unidirectionnel)** : Dans un *Decoder-Only*, on masque les positions futures dans la matrice d’attention, alors que dans un *Encoder-Only*, chaque token peut s’attendre sur tous les autres (bidirectionnel).
2. **Pas de cross-attention** : Dans BERT, il n’y en a pas car c’est un *Encoder-Only*, et dans ce GPT simplifié non plus, car il n’y a pas d’encodeur à interroger.
3. **Tâche typique** : Un *Decoder-Only Transformer* est généralement utilisé pour la génération autoregressive de texte. Chaque nouveau token est prédit en fonction des tokens précédents seulement.
4. **Embeddings** : GPT n’utilise pas (en général) de *segment embeddings*.

Cette implémentation reste simple et éducative ; dans la pratique, les modèles GPT (GPT-2, GPT-3, etc.) incluent davantage de subtilités (initialisations précises, utilisation systématique du *weight tying* avec l’embedding, optimisations, etc.).

> regardont l'implémenation du LLama3 [model](https://github.com/huggingface/transformers/blob/d3af76df58476830eb5b5981decc64af15e369f5/src/transformers/models/llama/modeling_llama.py#L750)

# RNN vs transformers

## RNN summary

**Advantages:**

1. **Sequential Processing:** RNNs are inherently designed for sequential data processing, making them perfect for time series prediction, natural language processing, and speech recognition.
2. **low cost inference:** RNNs tend to require fewer computational resources than Transformer models as they process input sequences step by step rather than in parallel.

**Disadvantages:**

1. **Vanishing and Exploding Gradient Problem:** During back-propagation in deep RNNs, gradients are multiplied by the weight matrix at every timestep. This can result in gradients that either explode or vanish, making it challenging to train deep RNNs.
2. **Long-term Dependencies:** RNNs struggle to learn long-term dependencies due to the vanishing gradient problem.
3. **Cannot Process in Parallel:** The sequential nature of RNNs means they cannot take advantage of modern GPUs which excel in performing parallel operations.

![triangle](./asset/rnn-vs-transformer.png)
